In [1]:
from src.chat import Chat
from app.models.UserInput import UserInput

In [2]:
chat = Chat()

/Users/pongsakon/Study/CEDT/2_semeter_1/data_sci/dsde-project/src/database/qdrant.py:37: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = pd.read_csv("combined_data.csv")


## Set System Prompt

In [3]:
user_message = "Give me the papers that related to field of aerodynamic especially the military purposes"
chat_id = "abc"

user_input = UserInput(
    chat_id=chat_id,
    message=user_message,
    currentGraph=None
)

# Goal
1. Give "Keyword node"
2. Give "GrapLink from GraphData"

In [ ]:
### FIX HERE
system_template_prompt = """You are an AI assistance for encouraging the system to initialize new "Node" from below data that was fetched from QdrantDB. Node have two types which are "Paper" and "Keyword".
Keyword is a short word that related to topic, title or field of the input messages from user while "Paper" is the main content of articles in the database. Your tasks are generating "Keyword" node based on summarizing the content of user messages and "Paper" node, a node that contains more . Both of them follow this structure of node object: 

Don't forget to ensure that the values that will be assigned are related to data in the databas

This is the guideines for you:
1. Analyze the given user input to find some "Keyword" that related to the user message topic, papers, articles.
2. From the paper that user give to you, You have to find the top most important 4-5 topics and keep their id, title, type, year, abstract (if the abstract too long just keep as long as you think it's important and meaningful),  author and source in Json format.
3. After you have done the step 2, Create the node, this is the example structure. 
    "links" : [
        
            "source" : "2410.18541_arxiv",
            "target" : "2204.13154_arxiv"
        ,
        
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2204.13154_arxiv"
        ,
        
            "source" : "transformer-and-attention-is-all-you-need",
            "target" : "2410.18541_arxiv"
        ,
    ]
Ensure that every pairs of the links member are related and as you can see from the example, some graph link might have the same source, which store
the ID of the starter node (the node that have higher level).
Don't for get to change the id of source and target to the correct id of provided paper. The IDs in the examples isn't always the same as what users provide to you.
However, You can adjust the inhertance of the graph using your decision to make it be more appropriate.
5. Node with type "keyword" will have 3 fields id, type and title (no label)
4. In the end, Return them in json format by nodes and links is in the same level and re-check if everything is correct and follows the guidelines, IDs in GraphLinks are paired with the IDs from provided papers.
"""
### FIX HERE

chat.set_system_template_prompt(system_template_prompt)

# Set Chat Template Prompt

In [21]:
### FIX HERE
chat_template_prompt = """
Below is the GraphData that was fetched from QdrantDB. You can use this data to generate new "Node" based on the user message.
{paper_data}
"""
### FIX HERE

chat.set_chat_template_prompt(chat_template_prompt)

In [22]:
### See the result of the chat

initial_result = chat.test_initial_chat(user_input=user_input)

================================ Human Message =================================


Below is the GraphData that was fetched from QdrantDB. You can use this data to generate new "Node" based on the user message.
{
    "id": "85128243165.0_scopus",
    "title": "Product development of ice cream stick from raw banana powder",
    "type": "paper",
    "year": 2022,
    "abstract": "This research was focused on using raw bananas in place of natural wooden ice cream sticks. The raw bananas were used in a powdered form and then processed to create a substitute for the wooden sticks. A research operation was carried out, experimenting with different proportions of banana powder in a randomized complete block experimental design (RCBD). The properties of interest for the ice cream sticks were hardness, fracturability, and aw of samples, as well as a sensory evaluation. The research showed that ice cream sticks which contained 50\u201380% raw banana powder with 40% sugar and 40 min of baking resu

In [7]:
chat.get_chat_history(chat_id=chat_id)

[HumanMessage(content='\nBelow is the GraphData that was fetched from QdrantDB. You can use this data to generate new "Node" based on the user message.\n{\n    "id": "912.3355_arxiv",\n    "title": "Performance and Stability of a Winged Vehicle in Ground Effect",\n    "type": "paper",\n    "year": 2009,\n    "abstract": "  Present work deals with the dynamics of vehicles which intentionally operate\\nin the ground proximity. The dynamics in ground effect is influenced by the\\nvehicle orientation with respect to the ground, since the aerodynamic force and\\nmoment coefficients, which in turn depend on height and angle of attack, also\\nvary with the Euler angles. This feature, usually neglected in the\\napplications, can be responsible for sizable variations of the aircraft\\nperformance and stability. A further effect, caused by the sink rate,\\ndetermines unsteadiness that modifies the aerodynamic coefficients. In this\\nwork, an analytical formulation is proposed for the force and m

In [23]:
print(initial_result.content)

```json
{
  "GraphNodes": [
    {
      "id": "aerodynamics",
      "type": "keyword",
      "title": "Aerodynamics"
    },
    {
      "id": "wing-design",
      "type": "keyword",
      "title": "Wing Design"
    },
    {
      "id": "flow-control",
      "type": "keyword",
      "title": "Flow Control"
    },
    {
      "id": "flight-mechanics",
      "type": "keyword",
      "title": "Flight Mechanics"
    },    
    {
      "id": "912.3355_arxiv",
      "type": "paper",
      "title": "Performance and Stability of a Winged Vehicle in Ground Effect",
      "year": 2009,
      "abstract": "Present work deals with the dynamics of vehicles operating in ground proximity.  Aerodynamic coefficients vary with Euler angles, impacting aircraft performance and stability.",
      "authors": ["Nicola de Divitiis"],
      "source": "arxiv"
    },
    {
      "id": "2209.0933_arxiv",
      "type": "paper",
      "title": "Simultaneous shape and topology optimization of wings",
      "year": 202

In [13]:
from typing import List
from app.models.GraphData import GraphLink
import json

In [18]:
def format_chat_json_to_graph_link(chat_json_response: str) -> List[GraphLink]:
    """
    Example of chat_json_response
    ```json
    [
        {
            "source": "some_id",
            "target": "some_id",
            "index": 0
        }
    ]
    ```
    """

    # Select content between ```json and ```
    chat_json_response = chat_json_response.split("```json")[1].split("```")[0]
    links_data = json.loads(chat_json_response)
    print(links_data)
    graph_links = [GraphLink(**link) for link in links_data]
    return graph_links

In [19]:
format_chat_json_to_graph_link(initial_result.content)

{'GraphNodes': [{'id': 'aerodynamics', 'type': 'keyword', 'title': 'Aerodynamics'}, {'id': 'wing-design', 'type': 'keyword', 'title': 'Wing Design'}, {'id': 'flight-mechanics', 'type': 'keyword', 'title': 'Flight Mechanics'}, {'id': '912.3355_arxiv', 'type': 'paper', 'title': 'Performance and Stability of a Winged Vehicle in Ground Effect', 'year': 2009, 'abstract': 'Present work deals with the dynamics of vehicles which intentionally operate in the ground proximity. The dynamics in ground effect is influenced by the vehicle orientation with respect to the ground, since the aerodynamic force and moment coefficients, which in turn depend on height and angle of attack, also vary with the Euler angles. This feature, usually neglected in the applications, can be responsible for sizable variations of the aircraft performance and stability.', 'authors': ['Nicola de Divitiis'], 'source': 'arxiv'}, {'id': '2209.0933_arxiv', 'type': 'paper', 'title': 'Simultaneous shape and topology optimizatio

TypeError: app.models.GraphData.GraphLink() argument after ** must be a mapping, not str

# Set Detect Additional Data

In [ ]:
user_detect_additional_data = UserInput(
    chat_id=chat_id,
    message="Yes, I have additional data",
    currentGraph=
)

In [ ]:
detect_additional_data_template = """
{user_message}

{current_graph}
"""
chat.set_detect_additional_data_template(detect_additional_data_template)